<a href="https://colab.research.google.com/github/zb15/B2BNetworkWiki/blob/main/functions/Function3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FUNCTION 3. visualise_b2b_network()

Currently the package has 3 main functions:            

1.   choose_company(input_name, search_option='all')
2.   get_companies_network(QIDs, num_runs=(5,5,5,5))
3.   visualise_b2b_network(df)

The **visualise_b2b_network()** function has 2 steps at the moment (that can easily be made into 1 function).
The 1st step (with clean_and_join(p_df, ob_df, s_df, oo_df) function) is to clean the 4 datasets and join them
together into a final_df. The wikidata requires a lot of cleaning, handling duplicates etc. and comments are included
within (the quite long) code. And the function that visualising the network from final_df dataframe (visualise_b2b_network()).


### importing libraries

In [ ]:
!pip install pyvis --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install pycountry --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install pycountry-convert --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
# Importing the modules
import requests
import random
import pandas as pd
import numpy as np

### Prepare dataset: including various functions such as is_human(), get_description() etc

In [ ]:
# Checking if entity is a human, then get description too
def is_human(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT DISTINCT
                                ?human
                                (REPLACE(STR(?human), "http://www.wikidata.org/entity/", "") AS ?QID)
                                ?humanLabel
                                ?humanDescription
      WHERE {{
            VALUES ?human {{wd:{items}}} .
            ?human wdt:P31/wdt:P279* wd:Q5 .
            ?article schema:about ?human .
            ?article schema:inLanguage "en" .
            ?article schema:isPartOf <https://en.wikipedia.org/>.
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            FILTER (LANG (?humanDescription) = "en")
            }}
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()

    # Initializing an empty dictionary to store the information
    info = {}
    # Looping through the results
    for result in data["results"]["bindings"]:
        # Adding the company's QID to the dictionary if exists
        if "QID" in result:
            info["QID"] = result["QID"]["value"]
        # Adding the company label to the dictionary if exists
        if "humanLabel" in result:
            info["name"] = result["humanLabel"]["value"]
        # Adding the industry label to the dictionary if exists
        if "humanDescription" in result:
            info["descr"] = result["humanDescription"]["value"]
    # Returning the dictionary of information
    return info



In [ ]:
# Getting description for the unknown items
def get_description(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT DISTINCT
                                ?item
                                (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                                ?itemLabel
                                ?itemDescription
      WHERE {{ wd:{items} rdfs:label ?itemLabel .
            OPTIONAL {{ wd:{items} schema:description ?itemDescription }} .
            BIND (wd:{items} AS ?item) .
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            FILTER (LANG (?itemDescription) = "en")
            }}
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()

    # Initializing an empty dictionary to store the information
    info = {}
    # Looping through the results
    for result in data["results"]["bindings"]:
        # Adding the company's QID to the dictionary if exists
        if "QID" in result:
            info["QID"] = result["QID"]["value"]
        # Adding the company label to the dictionary if exists
        if "itemLabel" in result:
            info["name"] = result["itemLabel"]["value"]
        # Adding the industry label to the dictionary if exists
        if "itemDescription" in result:
            info["descr"] = result["itemDescription"]["value"]
    # Returning the dictionary of information
    return info



In [ ]:
def clear_each_df(df):
  import pandas as pd
  # convert endtime column to string and filter out rows with valid dates
  if 'endtime' in df.columns:
    df['endtime'] = df['endtime'].astype(str)
    df1 = df[~df['endtime'].str.contains('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')]
  new_df =pd.DataFrame(columns=df.columns)
  # create a set of unique QID and obQID pairs
  qid1_qid2_pairs = set(zip(df1.iloc[:, 1], df1.iloc[:, 6]))
  # loop over the unique pairs and select the last row for each pair
  for qid1, qid2 in qid1_qid2_pairs:
      temp_df = df1.loc[(df1.iloc[:, 1] == qid1) & (df1.iloc[:, 6] == qid2)]
      temp_df = temp_df.drop_duplicates().reset_index(drop = True)
      if len(temp_df) == 1:
          new_row = temp_df.iloc[0]
          new_df = pd.concat([new_df, new_row.to_frame().T])
      if len(temp_df) >= 2:
          # check if proportionofLabel column is in temp_df
          if 'proportionofLabel' in temp_df:
              # check if proportionofLabel column has any value
              if any(temp_df['proportionofLabel']):
                  # use list comprehension to filter rows with proportionofLabel equal to 'authorised capital'
                  auth_cap_rows = [row for row in temp_df.itertuples() if row.proportionofLabel == 'authorised capital']
                  if auth_cap_rows:
                      # select the last row from the filtered list
                      new_row = auth_cap_rows[-1]
                      new_df = pd.concat([new_df, pd.Series(new_row._asdict()).to_frame().T])
          else:
              # return auth_cap_rows empty
              auth_cap_rows = []
          # use list comprehension to filter rows with pointoftime or starttime containing valid dates
          date_rows = [row for row in temp_df.itertuples() if '\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z' in (row.pointoftime or row.starttime)]
          if date_rows:
              # use max function to find the latest date from the filtered list
              latest_date = max(date_rows, key=lambda x: x.pointoftime or x.starttime)
              new_row = latest_date
              new_df = pd.concat([new_df, pd.Series(new_row._asdict()).to_frame().T])
  # reset index and drop duplicates
  new_df.reset_index(drop=True, inplace=True)
  new_df = new_df.drop_duplicates().reset_index(drop = True)

  # add those relations that has an endtime to the new dataframes
  if 'endtime' in df.columns:
    df['endtime'] = df['endtime'].astype(str)
    df2 = df[df['endtime'].str.contains('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')]
    df2.loc[:, 'proportion'] = 4
    df2['endtime'] = df2['endtime'].str[:10]
    df2 = df2.drop_duplicates()
  new_df = pd.concat([new_df, df2]).reset_index(drop = True)

  return new_df


In [ ]:
def drop_dupl_joined(new_df2):
  ### remove the duplicates from the joined datasets
  # Step 1. save the single parent-child pairs in the new3_df2
  #         if there are 2 or more of the same parent-child pair, then \
  #         drop the ones within these that has blank p_proportion.
  # Step 2. in a new list comprehension (temp_step2_df) save the ones that now became single parent-child.
  #         if there are 2 or more and if one of the p_prop value is == 2, then \
  ##        drop this row with value == 2
  #         Finally, if  there are 2 or more and their p_prop value is equal, then keep only the last
  ##        (this will allow more values to pass through if they differ)
  # STEP 1
  # Use a set to store the unique cQID values
  cQID_set = set(new_df2['cQID'])
  # Use a generator expression to iterate over the pQID values
  pQID_gen = (i for i in new_df2['pQID'])
  # Use a list comprehension to filter the rows based on pQID and cQID values
  temp_new3_df2 = [new_df2.loc[(new_df2['pQID'] == i) & (new_df2['cQID'] == j)]
                  for i in pQID_gen for j in cQID_set]
  # Use another list comprehension to process the filtered rows based on the
  # length and p_proportion values
  new3_df2 = pd.concat([row.iloc[0].to_frame().T if len(row) == 1
                        else row.dropna(subset=['p_proportion']).loc[row['p_proportion'].notnull()]
                        for row in temp_new3_df2])
  new3_df2 = new3_df2.drop_duplicates().reset_index(drop = True)
  # STEP 2
  # Use a set to store the unique cQID values
  cQID_set2 = set(new3_df2['cQID'])
  # Use a generator expression to iterate over the pQID values
  pQID_gen2 = (i for i in new3_df2['pQID'])
  # Use a list comprehension to filter the rows based on pQID and cQID values
  temp_step2_df = [new3_df2.loc[(new3_df2['pQID'] == i) & (new3_df2['cQID'] == j)]
                  for i in pQID_gen2 for j in cQID_set2]
  # Drop the rows with p_proportion value equal to 2
  final_df = pd.concat([row.iloc[0].to_frame().T if len(row) == 1 \
                        else row.loc[row['p_proportion'] != 2] for row in temp_step2_df])
  # Keep only the last row if there are multiple rows with the same pQID and cQID
  # values and equal p_proportion values
  final_df = final_df.drop_duplicates(subset=['pQID', 'cQID', 'p_proportion'], keep='last')
  final_df = final_df.drop_duplicates().reset_index(drop = True)
  # fill the blank p_proportion values with number 3, this will mean unknown on the graph
  final_df['p_proportion'] = final_df['p_proportion'].fillna(3)
  return final_df

In [ ]:
def top_industries(final_df):
  import numpy as np

  # create a dictionary of industry frequencies
  # split industry columns by ";" and create separate columns
  df_split1 = final_df["p_industries"].str.split(";", expand=True)
  df_split2 = final_df["c_industries"].str.split(";", expand=True)
  # join the two dataframes horizontally
  df_split = pd.concat([df_split1, df_split2], axis=1)
  # reshape the dataframe so that each expression is in a single column
  df_melt = df_split.melt(value_name="expression")
  # drop the variable column
  df_melt = df_melt.drop("variable", axis=1)
  # count the frequency of each expression and convert to dictionary
  freq = df_melt["expression"].value_counts().to_dict()

  # define a custom function to pick the expression with highest frequency
  def pick_expression(lst):
    import numpy as np
    # split the list by semicolon
    lst = lst.split(";")
    # initialize the best expression and frequency
    best_exp = None
    best_freq = 0
    # loop through the list
    for exp in lst:
        # get the frequency of the expression
        freq_exp = freq.get(exp, 0)
        # if the frequency is higher than the best frequency, update the best expression and frequency
        if freq_exp > best_freq:
            best_exp = exp
            best_freq = freq_exp
    # return the best expression
    return best_exp

  # apply the custom function to each cell in col2
  final_df["top_p_industries"] = final_df["p_industries"].apply(pick_expression)
  final_df["top_c_industries"] = final_df["c_industries"].apply(pick_expression)

  # replace the empty strings with NaN
  final_df["top_p_industries"] = final_df["top_p_industries"].replace("", np.nan)
  final_df["top_c_industries"] = final_df["top_c_industries"].replace("", np.nan)
  final_df["p_industries"] = final_df["p_industries"].replace("", np.nan)
  final_df["c_industries"] = final_df["c_industries"].replace("", np.nan)

  # use isna method to get a boolean mask of missing values
  mask = final_df["top_p_industries"].isna()
  mask2 = final_df["top_c_industries"].isna()
  mask3 = final_df["p_industries"].isna()
  mask4 = final_df["c_industries"].isna()

  # use loc method to assign "unknown" to the rows where col2 is missing
  final_df.loc[mask, "top_p_industries"] = "unknown"
  final_df.loc[mask2, "top_c_industries"] = "unknown"
  final_df.loc[mask3, "p_industries"] = "unknown"
  final_df.loc[mask4, "c_industries"] = "unknown"

  final_df["top_p_industries"] = final_df["top_p_industries"].str.lstrip()
  final_df["top_c_industries"] = final_df["top_c_industries"].str.lstrip()

  # make a list of all those parent QIDs where the industry is "unknown"
  unique_values1 = final_df.loc[(final_df['top_p_industries'] == 'unknown'), 'pQID'].drop_duplicates().values.tolist()
  # making sure only those QIDs in unique_values that starts with "Q"
  unique_values1 = [item for item in unique_values1 if item.startswith("Q")]
  # create a dataframe for the results
  p_human_df1 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results =[]
  # Loop through the values
  for id in unique_values1:
    # Run the module with the value as input and get the result
    result = pd.DataFrame(is_human(id), index=[0])
    # Appending the new dataframe to the list
    results.append(result)

  # join the new results to the dataframe
  p_human_df1 = pd.concat([p_human_df1] + results, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  #apply humans and their descriptions to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['pQID'] in p_human_df1['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index = p_human_df1[p_human_df1['QID'] == row['pQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value = p_human_df1.loc[match_index[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'p_industries'] = descr_value
          final_df.loc[index, 'top_p_industries'] = "human"

  # make a list of all those QIDs where the industry is "unknown"
  unique_values3 = final_df.loc[(final_df['top_p_industries'] == 'unknown'), 'pQID'].drop_duplicates().values.tolist()
  unique_values4 = final_df.loc[(final_df['top_c_industries'] == 'unknown'), 'cQID'].drop_duplicates().values.tolist()
  # making sure only those QIDs in unique_values that starts with "Q"
  unique_values3 = [item for item in unique_values3 if item.startswith("Q")]
  unique_values4 = [item for item in unique_values4 if item.startswith("Q")]
  # create a dataframe for the results
  p_item_df1 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results =[]
  # Loop through the values
  for id in unique_values3:
    # Run the module with the value as input and get the result
    result = pd.DataFrame(get_description(id), index=[0])
    # Appending the new dataframe to the list
    results.append(result)

  # join the new results to the dataframe
  p_item_df1 = pd.concat([p_item_df1] + results, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  # create a dataframe for the results
  c_item_df2 = pd.DataFrame(columns=["QID", "name", "descr"])
  #initialising a new list to store the results
  results2 =[]
  # Loop through the values
  for id2 in unique_values4:
    # Run the module with the value as input and get the result
    result2 = pd.DataFrame(get_description(id2), index=[0])
    # Appending the new dataframe to the list
    results2.append(result2)

  # join the new results to the dataframe
  c_item_df2 = pd.concat([c_item_df2] + results2, axis=0, join='outer').drop_duplicates().reset_index(drop = True)

  #apply descriptions for unknown parents to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['pQID'] in p_item_df1['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index = p_item_df1[p_item_df1['QID'] == row['pQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value = p_item_df1.loc[match_index[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'p_industries'] = descr_value
          final_df.loc[index, 'top_p_industries'] = "other"

  #apply descriptions for unknown children to new_df2 dataframe
  for index, row in final_df.iterrows():
      if row['cQID'] in c_item_df2['QID'].values:
          # get the index of the matching row in p_human_df1
          match_index2 = c_item_df2[c_item_df2['QID'] == row['cQID']].index
          # get the value from the "descr" column of p_human_df1
          descr_value2 = c_item_df2.loc[match_index2[0], 'descr']
          # update the values in new_df2
          final_df.loc[index, 'c_industries'] = descr_value2
          final_df.loc[index, 'top_c_industries'] = "other"

  # convert the values in columns to strings
  final_df['p_industries'] = final_df['p_industries'].astype(str)
  final_df['top_p_industries'] = final_df['top_p_industries'].astype(str)
  final_df['c_industries'] = final_df['c_industries'].astype(str)
  final_df['top_c_industries'] = final_df['top_c_industries'].astype(str)
  return final_df

In [ ]:
# a function to convert the known country names to the continents they belong
# should first install pycountry and pycountry_convert
def country_to_continent(country_name):
  import pycountry_convert as pc
  if country_name == 'unknown':
    return 'unknown'
  else:
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name


### prepare dataset main function: clean_join(p_df, ob_df, s_df, oo_df)

In [ ]:
# clean each dataset and then join them
def clean_join(p_df, ob_df, s_df, oo_df):
  #global final_df, new_p_df, new_ob_df, new_s_df, new_oo_df
  # convert the data columns to strings
  p_df.loc[:, ['pointoftime', 'starttime']] = p_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  ob_df.loc[:, ['pointoftime', 'starttime']] = ob_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  s_df.loc[:, ['pointoftime', 'starttime']] = s_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  oo_df.loc[:, ['pointoftime', 'starttime']] = oo_df.loc[:, ['pointoftime', 'starttime']].astype(str)
  ### remove the duplicates within p dataset - keep the last
  new_p_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'parent', \
                                    'pQID', 'parentLabel', 'parentcountryLabel', 'pindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_p_df = pd.concat([new_p_df, clear_each_df(p_df)]).reset_index(drop=True)

  ### remove the duplicates within ob dataset - keep the last
  new_ob_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownedby', \
                                    'obQID', 'ownedbyLabel', 'ownedbycountryLabel', 'obindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_ob_df = pd.concat([new_ob_df, clear_each_df(ob_df)]).reset_index(drop=True)

  ### remove the duplicates within s dataset - keep the last
  new_s_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'subsidiary', \
                                    'sQID', 'subsidiaryLabel', 'subsidiarycountryLabel', 'sindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_s_df = pd.concat([new_s_df, clear_each_df(s_df)]).reset_index(drop=True)

  ### remove the duplicates within oo dataset - keep the last
  new_oo_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownerof', \
                                    'ooQID', 'ownerofLabel', 'ownerofcountryLabel', 'ooindustries', 'proportion', \
                                    'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  new_oo_df = pd.concat([new_oo_df, clear_each_df(oo_df)]).reset_index(drop=True)

  ### join together the dataframes into a new_df
  new_df = pd.DataFrame(columns=["pQID", "parent", "parent_country", "p_industries", "cQID", "child", \
                                "child_country", "c_industries", "p_proportion", "proportionofLabel", \
                                "pointoftime", "starttime", "endtime"])
  # Create an empty list to store the new rows
  new_p_rows = []
  # Loop over the new_p_df dataframe
  for i in range(len(new_p_df)):
      # Create a new row as a dictionary
      new_p_row = {'pQID': new_p_df['pQID'][i], 'parent': new_p_df['parentLabel'][i], 'parent_country': new_p_df['parentcountryLabel'][i], \
                'p_industries': new_p_df['pindustries'][i], 'cQID': new_p_df['QID'][i], \
                'child': new_p_df['itemLabel'][i], 'child_country': new_p_df['itemcountryLabel'][i], 'c_industries': new_p_df['industries'][i], \
                'p_proportion':new_p_df['proportion'][i], 'proportionofLabel':new_p_df['proportionofLabel'][i], \
                'pointoftime':new_p_df['pointoftime'][i], 'starttime':new_p_df['starttime'][i], 'endtime': new_p_df['endtime'][i]}
      # Convert the dictionary to a dataframe and append it to the list
      new_p_rows.append(pd.DataFrame(new_p_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_p_rows, ignore_index=True)
  new_df['p_proportion'] = new_df['p_proportion'].fillna(2)
  # Create an empty list to store the new rows
  new_ob_rows = []
  for i in range(len(new_ob_df)):
      new_ob_row = {'pQID': new_ob_df['obQID'][i], 'parent': new_ob_df['ownedbyLabel'][i], 'parent_country': new_ob_df['ownedbycountryLabel'][i], \
                'p_industries': new_ob_df['obindustries'][i], 'cQID': new_ob_df['QID'][i], \
                'child': new_ob_df['itemLabel'][i], 'child_country': new_ob_df['itemcountryLabel'][i], 'c_industries': new_ob_df['industries'][i], \
                'p_proportion':new_ob_df['proportion'][i], 'proportionofLabel':new_ob_df['proportionofLabel'][i], \
                'pointoftime':new_ob_df['pointoftime'][i], 'starttime':new_ob_df['starttime'][i], 'endtime': new_ob_df['endtime'][i]}
      new_ob_rows.append(pd.DataFrame(new_ob_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_ob_rows, ignore_index=True)
  # Create an empty list to store the new rows
  new_s_rows = []
  for i in range(len(new_s_df)):
      new_s_row = {'pQID': new_s_df['QID'][i], 'parent': new_s_df['itemLabel'][i], 'parent_country': new_s_df['itemcountryLabel'][i], \
                'p_industries': new_s_df['industries'][i], 'cQID': new_s_df['sQID'][i], \
                'child': new_s_df['subsidiaryLabel'][i], 'child_country': new_s_df['subsidiarycountryLabel'][i], 'c_industries': new_s_df['sindustries'][i], \
                'p_proportion':new_s_df['proportion'][i], 'proportionofLabel':new_s_df['proportionofLabel'][i], \
                'pointoftime':new_s_df['pointoftime'][i], 'starttime':new_s_df['starttime'][i], 'endtime': new_s_df['endtime'][i]}
      new_s_rows.append(pd.DataFrame(new_s_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_s_rows, ignore_index=True)
  # Create an empty list to store the new rows
  new_oo_rows = []
  for i in range(len(new_oo_df)):
      new_oo_row = {'pQID': new_oo_df['QID'][i], 'parent': new_oo_df['itemLabel'][i], 'parent_country': new_oo_df['itemcountryLabel'][i], \
                'p_industries': new_oo_df['industries'][i], 'cQID': new_oo_df['ooQID'][i], \
                'child': new_oo_df['ownerofLabel'][i], 'child_country': new_oo_df['ownerofcountryLabel'][i], 'c_industries': new_oo_df['ooindustries'][i], \
                'p_proportion':new_oo_df['proportion'][i], 'proportionofLabel':new_oo_df['proportionofLabel'][i], \
                'pointoftime':new_oo_df['pointoftime'][i], 'starttime':new_oo_df['starttime'][i], 'endtime': new_oo_df['endtime'][i]}
      new_oo_rows.append(pd.DataFrame(new_oo_row, index=[0]))
  # Concatenate the list of dataframes with the original dataframe
  new_df = pd.concat([new_df] + new_oo_rows, ignore_index=True)
  new_df2 = new_df.drop_duplicates().reset_index(drop = True)

  # drop the duplicates from the joined dataframe with various rules defined by drop_dupl_joined() function
  final_df = drop_dupl_joined(new_df2)
  # clear up industries and find the top ones for classification. unknown ones: get description (also checks if its human)
  final_df = top_industries(final_df)

  # replace missing country values with "unknown"
  final_df["parent_country"] = final_df["parent_country"].replace("", np.nan)
  final_df["child_country"] = final_df["child_country"].replace("", np.nan)
  mask_p = final_df["parent_country"].isna()
  mask_c = final_df["child_country"].isna()
  final_df.loc[mask_p, "parent_country"] = "unknown"
  final_df.loc[mask_c, "child_country"] = "unknown"
  # apply the country_to_continent function to get the continents the companies are located
  final_df['parent_continent'] = final_df['parent_country'].apply(country_to_continent)
  final_df['child_continent'] = final_df['child_country'].apply(country_to_continent)
  return final_df


### function 3: visualise_b2b_network(final_df)

In [ ]:
def visualise_b2b_network(final_df):
  # import packages
  import pandas as pd
  from pyvis.network import Network
  import seaborn as sns
  import colorsys
  from collections import Counter
  import json

  # create network object
  net = Network(
      notebook = True,
      directed = True,            # directed graph
      bgcolor = "snow",          # background color of graph
      font_color = "navy",        # use navy for node labels
      cdn_resources = 'in_line',  # make sure Jupyter notebook can display correctly
      height = "1000px",          # height of chart
      width = "100%",             # fill the entire width
      select_menu=True,           # user can choose from campany name list
      filter_menu=True,           # user can search for colors - indicators for industries
      neighborhood_highlight=True, # clicking on a node highlights its connections and grays out others
      )

  # create lists of nodes and edges from dataframe columns
  nodes = []
  edges = []
  for parent, child, p_industry, c_industry, top_p_industry, top_c_industry, parent_country, parent_continent, \
      child_country, child_continent, proportion, endtime in zip(final_df['parent'], \
                                                                 final_df['child'], \
                                                                 final_df['p_industries'], \
                                                                 final_df['c_industries'], \
                                                                 final_df['top_p_industries'], \
                                                                 final_df['top_c_industries'], \
                                                                 final_df['parent_country'], \
                                                                 final_df['parent_continent'], \
                                                                 final_df['child_country'], \
                                                                 final_df['child_continent'], \
                                                                 final_df['p_proportion'], \
                                                                 final_df['endtime']):
      nodes.append((parent, p_industry, top_p_industry, parent_country, parent_continent))
      nodes.append((child, c_industry, top_c_industry, child_country, child_continent))
      edges.append((parent, child, proportion, endtime))

  # remove duplicate nodes
  #nodes = list(set(nodes))

  # create a set of unique industries
  industries = set(final_df['top_p_industries']).union(set(final_df['top_c_industries']))

  # get the number of unique industries
  n_colors = len(industries)

  # create a dictionary that maps each unique industry to a pastel color
  pastel_colors = sns.color_palette('muted', n_colors)

  # define a function that converts an rgb tuple to an rgba string with a pastel factor
  def rgb_to_rgba(rgb, pastel_factor):
      # convert rgb values to hls values
      h, l, s = colorsys.rgb_to_hls(*rgb)
      # increase lightness by multiplying with pastel factor
      l *= pastel_factor
      # convert hls values back to rgb values
      r, g, b = colorsys.hls_to_rgb(h, l, s)
      # normalize rgb values to be between 0 and 255
      r = int(r * 255)
      g = int(g * 255)
      b = int(b * 255)
      # add alpha value of 0.7
      a = 0.7
      # format rgba values as a string
      rgba = f'rgba({r}, {g}, {b}, {a})'
      return rgba

  group_color_map = {}
  for i, top_industry in enumerate(industries):
      # use the function with pastel_colors[i] and a pastel factor of 1.2
      group_color_map[top_industry] = rgb_to_rgba(pastel_colors[i], 1.2)

  # create a dictionary that maps each node to its degree
  node_degree_map = Counter()
  for edge in edges:
      node_degree_map[edge[0]] += 1
      node_degree_map[edge[1]] += 1

  # define a function that takes a degree and a scaling factor and returns a size for the node
  def degree_to_size(degree, scaling_factor):
      # multiply degree by scaling factor to get base size
      base_size = degree * scaling_factor
      # add minimum size to base size to ensure no node is too small
      min_size = 3
      final_size = base_size + min_size
      # return final size as an integer
      return int(final_size)

  # create a dictionary that maps each node to its size
  node_size_map = {}
  for node in node_degree_map:
      # use the function with node_degree_map[node] and a scaling factor of 5
      node_size_map[node] = degree_to_size(node_degree_map[node], 5)

  # define a different color for the highlighted nodes
  highlight_color = 'green'
  #node1 = root_companies[0]
  #node2 = root_companies[1]

  # create a set of unique countries and continents
  countries = set(final_df['parent_country']).union(set(final_df['child_country']))
  continents = set(final_df['parent_continent']).union(set(final_df['child_continent']))

  # add nodes and edges to network object
  for node, industry, top_industry, countries, continents in nodes:
      color = group_color_map[top_industry]
      # check if the node name is equal to node1 or node2 and use highlight_color if so
      #if node == root_companies:
      #    color = highlight_color
      #else:
      #    color = group_color_map[top_industry]
      # assign shapes for teh continents
      if continents == 'North America':
        node_shape = 'triangle'
      elif continents == 'South America':
        node_shape = 'triangleDown'
      elif continents == 'Europe':
        node_shape = 'star'
      elif continents == 'Africa':
        node_shape = 'diamond'
      elif continents == 'Asia':
        node_shape = 'square'
      elif continents == 'Australia':
        node_shape = 'ellipse'
      else:
        node_shape = 'dot'
      size = node_size_map[node]
      net.add_node(node, label=node, group=top_industry, color=color, shape=node_shape, title=node + ', '+ countries + ' (' + industry + ')', value=size)

  for e in edges:
      if e[2] > 0 and e[2] < 0.5:
          net.add_edge(e[0], e[1], title=str(e[2]), arrows={"to": True}, color='turquoise') #value=(e[2]/100), dashes=[6,10,1,10]
      elif e[2] >= 0.5 and e[2] <= 1:
          net.add_edge(e[0], e[1], title=str(e[2]), arrows={"to": True}, color='violet') #value=(e[2]/10),  dashes=False
      elif e[2] == 2:
          net.add_edge(e[0], e[1], title="parent", arrows={"to": True}, color='salmon') #dashes=[5,5]
      elif e[2] == 3:
          net.add_edge(e[0], e[1], title="unknown value", arrows={"to": True}, color='lime') # dashes=[1,10]
      elif e[2] == 4:
          net.add_edge(e[0], e[1], title="end date: "+str(e[3]), arrows={"to": True}, color='grey')


  net.repulsion(
      node_distance=100,
      central_gravity=0.2,
      spring_length=200,
      spring_strength=0.05,
      damping=0.09,
  )

  net.show_buttons(filter_='physics')
  #net.show_buttons(filter_=['nodes', 'edges', 'physics'])

  # show network graph
  return net.show('B2B_network_Wiki.html')